In [4]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('dataset_tratado.csv')

x = df.drop('INF_ANAM', axis=1)
y = df['INF_ANAM']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

train_data = lgb.Dataset(x_train, label=y_train)
test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',  
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',
    'num_leaves': 35,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

print(classification_report(y_test, y_pred_binary, zero_division=0))

[LightGBM] [Info] Number of positive: 509, number of negative: 851
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.441345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 502
[LightGBM] [Info] Number of data points in the train set: 1360, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.374265 -> initscore=-0.513964
[LightGBM] [Info] Start training from score -0.513964
              precision    recall  f1-score   support

           0       0.73      0.78      0.76       213
           1       0.34      0.50      0.40        76
           2       0.00      0.00      0.00        35
           3       0.00      0.00      0.00        16

    accuracy                           0.60       340
   macro avg       0.27      0.32      0.29       340
weighted avg       0.53      0.60      0.56       340

